In [53]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
from sklearn.model_selection import train_test_split
import sklearn

In [54]:
customers = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/customers.csv')
transactions = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/transactions_train.csv')
articles = pd.read_csv('/kaggle/input/h-and-m-personalized-fashion-recommendations/articles.csv')

In [55]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id
0,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001,0.050831,2
1,2018-09-20,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023,0.030492,2
2,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004,0.015237,2
3,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003,0.016932,2
4,2018-09-20,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004,0.016932,2


In [56]:
customers.head()

,customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,NaN,NaN,ACTIVE,NONE,49.0,52043ee2162cf5aa7ee79974281641c6f11a68d276429a...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...,NaN,NaN,ACTIVE,NONE,25.0,2973abc54daa8a5f8ccfe9362140c63247c5eee03f1d93...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,NaN,NaN,ACTIVE,NONE,24.0,64f17e6a330a85798e4998f62d0930d14db8db1c054af6...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...,NaN,NaN,ACTIVE,NONE,54.0,5d36574f52495e81f019b680c843c443bd343d5ca5b1c2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...,1.0,1.0,ACTIVE,Regularly,52.0,25fa5ddee9aac01b35208d01736e57942317d756b32ddd...


In [57]:
articles.head()

,article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,...,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
0,108775015,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,9,Black,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
1,108775044,108775,Strap top,253,Vest top,Garment Upper body,1010016,Solid,10,White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
2,108775051,108775,Strap top (1),253,Vest top,Garment Upper body,1010017,Stripe,11,Off White,...,Jersey Basic,A,Ladieswear,1,Ladieswear,16,Womens Everyday Basics,1002,Jersey Basic,Jersey top with narrow shoulder straps.
3,110065001,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,9,Black,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."
4,110065002,110065,OP T-shirt (Idro),306,Bra,Underwear,1010016,Solid,10,White,...,Clean Lingerie,B,Lingeries/Tights,1,Ladieswear,61,Womens Lingerie,1017,"Under-, Nightwear","Microfibre T-shirt bra with underwired, moulde..."


In [58]:
customers.drop(['FN', 'Active', 'club_member_status', 'fashion_news_frequency', 'age', 'postal_code'], inplace=True, axis=1)

In [59]:
customers.head()

,customer_id
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...
1,0000423b00ade91418cceaf3b26c6af3dd342b51fd051e...
2,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...
3,00005ca1c9ed5f5146b52ac8639a40ca9d57aeff4d1bd2...
4,00006413d8573cd20ed7128e53b7b13819fe5cfc2d801f...


In [60]:
customer_id = customers.values.flatten().astype(str)

In [61]:
transactions.drop(['t_dat', 'price', 'sales_channel_id'], inplace=True, axis=1)

In [62]:
transactions.head()

,customer_id,article_id
0,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,663713001
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,541518023
2,00007d2de826758b65a93dd24ce629ed66842531df6699...,505221004
3,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687003
4,00007d2de826758b65a93dd24ce629ed66842531df6699...,685687004


In [63]:
train, test = sklearn.model_selection.train_test_split(transactions, train_size = 0.7, random_state=42)


In [64]:
articles.drop(['product_code','prod_name', 'product_type_no', 'product_type_name', 'product_group_name', 'graphical_appearance_no', 'graphical_appearance_name', 'colour_group_code', 'colour_group_name', 
        'perceived_colour_value_id', 'perceived_colour_value_name','perceived_colour_master_id', 
        'perceived_colour_master_name', 'department_no', 'department_name', 'index_code', 'index_name', 'index_group_no', 'index_group_name',
        'section_no', 'section_name', 'garment_group_name', 'garment_group_no', 'detail_desc'], inplace=True, axis=1)

In [65]:
articles.head()

,article_id
0,108775015
1,108775044
2,108775051
3,110065001
4,110065002


In [66]:
article_id = articles.values.flatten().astype(int)

In [67]:
customer_embedding = tf.keras.layers.Embedding(len(customer_id), 12)
article_embedding = tf.keras.layers.Embedding(len(article_id), 12)

In [68]:
article_table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(tf.constant(article_id, dtype=tf.int32), range(len(article_id))), -1)

In [69]:
article_table.lookup(tf.constant([110065002]))

<tf.Tensor: shape=(1,), dtype=int32, numpy=array([4], dtype=int32)>

In [112]:
class SimpleRecommender(tf.keras.Model):
    def __init__(self, customer_id, article_id, length_of_embedding):
        super(SimpleRecommender, self).__init__()
        self.article_id = tf.constant(article_id, dtype=tf.int32)
        self.customer_id = tf.constant(customer_id, dtype=tf.string)
        self.customer_table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(self.customer_id, range(len(customer_id))), -1)
        self.article_table = tf.lookup.StaticHashTable(tf.lookup.KeyValueTensorInitializer(self.article_id, range(len(article_id))), -1)
        
        self.customer_embedding = tf.keras.layers.Embedding(len(customer_id), length_of_embedding)
        self.article_embedding = tf.keras.layers.Embedding(len(article_id), length_of_embedding)
        
        self.dot= tf.keras.layers.Dot(axes=-1)
        
    def call(self, inputs):
        user = inputs[0]
        products = inputs[1]
        
        user_embedding_index = self.customer_table.lookup(user)
        product_embedding_index = self.article_table.lookup(products)
        
        user_embedding_values = self.customer_embedding(user_embedding_index)
        product_embedding_values = self.article_embedding(product_embedding_index)
        
        return tf.squeeze(self.dot([user_embedding_values, product_embedding_values]), 1)
    
    @tf.function
    def call_item_item(self, article_id):
        product_x = self.article_table.lookup(article_id)
        pe = tf.expand_dims(self.article_embedding(product_x), 0)
        
        all_pe = tf.expand_dims(self.article_embedding.embeddings, 0)#note this only works if the layer has been built!
        scores = tf.reshape(self.dot([pe, all_pe]), [-1])
        
        top_scores, top_indices = tf.math.top_k(scores, k=100)
        top_ids = tf.gather(self.article_id, top_indices)
        return top_ids, top_scores

In [71]:
customer_tensor = tf.constant(transactions[["customer_id"]].values, dtype=tf.string)
article_tensor = tf.constant(transactions[["article_id"]].values, dtype=tf.int32)

dataset = tf.data.Dataset.from_tensor_slices((customer_tensor, article_tensor))
for x, y in dataset:
    print(x)
    print(y)
    break

tf.Tensor([b'000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318'], shape=(1,), dtype=string)
tf.Tensor([663713001], shape=(1,), dtype=int32)


In [72]:
random_negatives_indexes = tf.random.uniform((7, ), minval=0, maxval=len(article_id), dtype=tf.int32)
random_negatives_indexes

<tf.Tensor: shape=(7,), dtype=int32, numpy=
array([ 24293,  68294,  70555,  42940,  35970,  25464, 102182],
      dtype=int32)>

In [73]:
tf.gather(articles, random_negatives_indexes)


<tf.Tensor: shape=(7, 1), dtype=int64, numpy=
array([[608850001],
       [753802003],
       [760961003],
       [674611001],
       [648994006],
       [613020004],
       [902528007]])>

In [74]:
class Mapper():
    
    def __init__(self, possible_products, num_negative_products):
        self.num_possible_products = len(possible_products)
        self.possible_products_tensor = tf.constant(possible_products, dtype=tf.int32)
        
        self.num_negative_products = num_negative_products
        self.y = tf.one_hot(0, num_negative_products+1)
    
    def __call__(self, user, product):
        random_negatives_indexes = tf.random.uniform((self.num_negative_products, ), minval=0, maxval=self.num_possible_products, dtype=tf.int32)
        negatives = tf.gather(self.possible_products_tensor, random_negatives_indexes)
        candidates = tf.concat([product, negatives], axis = 0)
        return (user, candidates), self.y

In [75]:
dataset = tf.data.Dataset.from_tensor_slices((customer_tensor, article_tensor)).map(Mapper(article_id, 10))
for (u, c), y in dataset:
  print(u)
  print(c)
  print(y)
  break

tf.Tensor([b'000058a12d5b43e67d225668fa1f8d618c13dc232df0cad8ffe7ad4a1091e318'], shape=(1,), dtype=string)
tf.Tensor(
[663713001 817472003 777143002 662879004 703199004 919438001 702960002
 663774002 918948001 427114014 838865001], shape=(11,), dtype=int32)
tf.Tensor([1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.], shape=(11,), dtype=float32)


In [76]:
def get_dataset(df, article_id, num_negative_products):
    customer_tensor = tf.constant(df[["customer_id"]].values, dtype=tf.string)
    article_tensor = tf.constant(df[["article_id"]].values, dtype=tf.int32)

    dataset = tf.data.Dataset.from_tensor_slices((customer_tensor, article_tensor))
    dataset = dataset.map(Mapper(article_id, num_negative_products))
    dataset = dataset.batch(1024)
    
    return dataset

In [77]:
for (u, c), y in get_dataset(train, article_id, 4):
    print(u)
    print(c)
    print(y)
    break

tf.Tensor(
[[b'301302814fdc08d798a2ce004d94fec95721344cd57c8406e0bc8f187cef2814']
 [b'198d7b4358f597153bd44dbda417ce7aae23bb6d4e8a6114170728914e762fe6']
 [b'4f11ca62870f508c1aa75adf5bcd5ba347b1dac40dce67b3a6c7521b789c9b61']
 ...
 [b'0e63d0320f921843e075bb4eff48970eacae2ea5cdf2415461cc183c9df07157']
 [b'31e23642c2d513c45cadae517e4a83d87636cfde205958381f1a0bd70faed429']
 [b'da39197333e43d95c1398592d67313f9fbbfd3e572ecc41393d41448a85b8360']], shape=(1024, 1), dtype=string)
tf.Tensor(
[[745199002 672748001 617245002 694676004 754267038]
 [621381012 644514001 883026003 485726002 769601001]
 [744276001 694519001 609010001 705401001 865981006]
 ...
 [554141021 825649001 805141001 603916001 636093001]
 [706016007 695033001 717679010 372576017 783707012]
 [802842001 767925006 849738002 878333002 890847002]], shape=(1024, 5), dtype=int32)
tf.Tensor(
[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 ...
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]], shape=(1024, 5), dtype=float32)


In [78]:
# detect and init the TPU
#tpu = tf.distribute.cluster_resolver.TPUClusterResolver.connect()

# instantiate a distribution strategy
#tpu_strategy = tf.distribute.experimental.TPUStrategy(tpu)

# instantiating the model in the strategy scope creates the model on the TPU
#with tpu_strategy.scope():
    #model = tf.keras.Sequential(SimpleRecommender(customer_id, article_id, 15)) # define your model normally
    #model.compile(loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True), optimizer = tf.keras.optimizers.SGD(learning_rate=100.),
             #metrics = [tf.keras.metrics.CategoricalAccuracy()])
    
model = SimpleRecommender(customer_id, article_id, 15)
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(from_logits=True), optimizer = tf.keras.optimizers.SGD(learning_rate=100.),
             metrics = [tf.keras.metrics.CategoricalAccuracy()])

model.fit(get_dataset(train, article_id, 100), epochs=2)

Epoch 1/2
21731/21731 [==============================] - 3222s 148ms/step - loss: 3.7915 - categorical_accuracy: 0.1585
Epoch 2/2
21731/21731 [==============================] - 3146s 145ms/step - loss: 2.8605 - categorical_accuracy: 0.2888


In [110]:
test_product = 110065001

In [113]:
print("Recs for item {}: {}".format(test_product, model.call_item_item(tf.constant(test_product, dtype=tf.int32))))


AttributeError: in user code:

    File "/tmp/ipykernel_27/831477863.py", line 28, in call_item_item  *
        product_x = self.article_table.lookup(article_id)

    AttributeError: 'numpy.dtype[int64]' object has no attribute 'base_dtype'
